총점 및 순위 계산 

In [168]:
data = '부산_주거인구+무인회수기+주거지역.csv'

# 파일 불러오기
import pandas as pd

df = pd.read_csv(f'{data}', encoding='cp949')
df.columns

Index(['gid', 'lbl', 'val', 'lon', 'lat', '무인회수기', '주거지역'], dtype='object')

score_col에 점수로 환산할 대상 칼럼 지정

In [169]:
score_col = ['val', '무인회수기', '주거지역']
weight = [1]*len(score_col)

가중치 선언

In [174]:
weight = [1, -1, 1]

if (len(weight) != len(score_col)):
    print("입력한 가중치 갯수가 점수칼럼 갯수와 다르므로 확인후 다시 입력 바랍니다")
else:
    print("다음 줄을 실행 하세요")

다음 줄을 실행 하세요


In [175]:
# df = norm.copy()
# df.sort_values('주거지역')

In [176]:
# df.sum()

In [178]:
df = pd.read_csv(f'{data}', encoding='cp949', engine='python')
# df.set_index('gid', inplace=True)

################## QGIS에서 작업한 전처리 결과 CSV 읽기 ####################################################
pre_process_set = pd.read_csv(f'{data}', encoding='cp949', engine='python')
pre_process_set = pre_process_set.sort_values(by='gid')

# Null 이 있을 수 있는 칼럼은 Null을 0로 변경
pre_process_set.fillna(0, inplace=True)
##################  QGIS에서 작업한 전처리 결과 CSV 읽기 끝###################################################

################## MinMax정규화 ##########################################################################
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# scaler = StandardScaler()
scaler = MinMaxScaler()

# 점수칼럼 정규화
norm = pre_process_set.copy()
norm.fillna(0, inplace=True)
norm.set_index('gid', inplace=True)

norm[score_col] = scaler.fit_transform(norm.loc[:,score_col])
norm.reset_index(inplace=True)
################## 정규화 끝################################################################################


# 총점 계산
df = norm.copy()

for i in range(0, len(score_col)):
    name = score_col[i]
    df[name] = df[name] * weight[i] 

# df.set_index(['gid','주차장','lon','lat'], inplace=True)
df.set_index(['gid', 'lbl', 'lon', 'lat'], inplace=True)

df['sum'] = df.sum(1)
df.reset_index(inplace=True)

# 등수 계산 
df['순위'] = df['sum'].rank(method='dense', ascending=False).astype(int)

final = df.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

# 상위 20개만 학인
display("결과", final.head(20))

# 최종 결과 CSV 파일쓰기
final.to_csv('결과_총점.csv', index=False, sep=',', encoding='cp949')

'결과'

,gid,lbl,lon,lat,val,무인회수기,주거지역,sum,순위
0,留덈씪3880,25321.0,129.019759,35.110150,0.759546,-0.0,0.824878,1.584425,1
1,留덈씪4183,17969.0,129.053181,35.136775,0.539011,-0.0,1.000000,1.539011,2
2,留덈씪4082,19768.0,129.042038,35.127901,0.592975,-0.0,0.798700,1.391675,3
3,留덈씪4184,20095.0,129.053352,35.145789,0.602784,-0.0,0.744900,1.347683,4
4,留덈씪3980,19011.0,129.030729,35.110012,0.570267,-0.0,0.754829,1.325097,5
5,留덈씪3879,21988.0,129.019592,35.101136,0.659567,-0.0,0.633869,1.293436,6
6,留덈씪4592,17825.0,129.098660,35.217327,0.534691,-0.0,0.708070,1.242761,7
7,留덈씪4290,28156.0,129.065363,35.199730,0.844587,-0.0,0.326954,1.171541,8
8,留덈씪4178,9035.0,129.052326,35.091705,0.271020,-0.0,0.876693,1.147713,9
9,留덈씪4388,29402.0,129.075996,35.181560,0.881963,-0.0,0.259253,1.141216,10
